In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow import keras
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd 
import nltk
import numpy as np 
from nltk.corpus import stopwords 
import re
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 
import pickle
from nltk.stem import WordNetLemmatizer


In [17]:
data=pd.read_csv('tweet_emotions.csv')
data

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...
...,...,...,...
39995,1753918954,neutral,@JohnLloydTaylor
39996,1753919001,love,Happy Mothers Day All my love
39997,1753919005,love,Happy Mother's Day to all the mommies out ther...
39998,1753919043,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [18]:
nltk.download('stopwords')
STOPWORDS = stopwords.words("english")
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def clean(text):
    text = text.lower() 
    text=re.sub(r'@\w+',"", text)
    text = re.sub("[^\w\s]"," ",text) # Remove punctuations 
    text = " ".join(w for w in text.split() if w not in STOPWORDS)
    text = " ".join(lemmatizer.lemmatize(w) for w in text.split())
    return text
data['text'] = data['content'].apply(lambda x : clean(x)) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SiYu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SiYu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
Le = LabelEncoder()
y =  Le.fit_transform(np.array(data['sentiment']))
data['Y_Encoder']=y

data=data[['text','Y_Encoder']]

In [20]:
data

,text,Y_Encoder
0,know listenin bad habit earlier started freaki...,2
1,layin n bed headache ughhhh waitin call,10
2,funeral ceremony gloomy friday,10
3,want hang friend soon,3
4,want trade someone houston ticket one,8
...,...,...
39995,,8
39996,happy mother day love,7
39997,happy mother day mommy woman man long momma so...,7
39998,wassup beautiful follow peep new hit single ww...,5


In [21]:
X=data['text'].array
y=data['Y_Encoder']
from imblearn.over_sampling import SMOTEN
sampler = SMOTEN(random_state=0)
X_res, y_res = sampler.fit_resample(X.reshape(-1, 1), y)

In [22]:
X_res.shape

(112294, 1)

In [23]:
corpus = [''.join(ele) for ele in X_res]

In [24]:
voc_size=5000
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2462, 2975, 3900, 4605, 1109, 3236, 999, 2381],
 [579, 231, 4964, 4346, 2467, 1971, 2227],
 [3311, 3844, 1899, 1865],
 [1848, 689, 3010, 1920],
 [1848, 4841, 986, 4250, 513, 1237],
 [2675, 2064, 1673, 3225, 3505, 88, 3010],
 [2861, 3991, 3730, 448, 3010, 1848, 4452, 84, 608, 1848, 1815, 4543],
 [4699, 787, 435, 1841, 2750],
 [3231, 1631, 3331],
 [1856, 1037, 1865],
 [3889, 3472, 2022],
 [1490, 3422],
 [200, 4831, 2348, 4084, 1802, 4178, 2621],
 [1579, 3288, 4320, 172, 1579, 1432, 3645, 630, 630, 1538, 3288, 2375],
 [996, 3599],
 [1426, 4589, 3302],
 [4622],
 [3754, 3555, 1538, 107],
 [164, 3514, 1064, 4206, 1236, 3586, 2462],
 [2720, 4368, 874, 2723, 407, 84, 2592, 2440, 1253, 3010],
 [988, 3900, 3023, 1802, 525, 2861, 439, 3748],
 [4744,
  1158,
  175,
  3734,
  990,
  4084,
  2914,
  2402,
  989,
  1255,
  4697,
  988,
  84,
  989,
  2575],
 [1299,
  845,
  4418,
  3785,
  1276,
  4549,
  4372,
  1178,
  1006,
  787,
  4774,
  3991,
  1362,
  2590,
  4366],
 [2870, 4125, 2462, 2190

In [25]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3236  999 2381]
 [   0    0    0 ... 2467 1971 2227]
 [   0    0    0 ... 3844 1899 1865]
 ...
 [   0    0    0 ... 3603  373 1165]
 [   0    0    0 ... 3603  373 1165]
 [   0    0    0 ... 3603  373 1165]]


In [31]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128))
model.add(Dense(13,activation='softmax'))

opt = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_4 (LSTM)               (None, 20, 128)           86528     
                                                                 
 lstm_5 (LSTM)               (None, 128)               131584    
                                                                 
 dense_2 (Dense)             (None, 13)                1677      
                                                                 
Total params: 419,789
Trainable params: 419,789
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
X_final=np.array(embedded_docs)
y_final=np.array(y_res)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [29]:
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_train_label = label_binarizer.fit_transform(y_train)
y_test_label = label_binarizer.fit_transform( y_test)

In [32]:
model.fit(X_train,y_train_label,validation_data=(X_test,y_test_label),epochs=10,batch_size=80)

Epoch 1/10
1123/1123 [==============================] - 69s 58ms/step - loss: 1.3733 - accuracy: 0.5406 - val_loss: 0.9828 - val_accuracy: 0.6381
Epoch 2/10
1123/1123 [==============================] - 64s 57ms/step - loss: 0.8712 - accuracy: 0.7076 - val_loss: 0.8352 - val_accuracy: 0.7164
Epoch 3/10
1123/1123 [==============================] - 65s 58ms/step - loss: 0.8075 - accuracy: 0.7220 - val_loss: 0.8140 - val_accuracy: 0.7222
Epoch 4/10
1123/1123 [==============================] - 69s 62ms/step - loss: 0.7794 - accuracy: 0.7319 - val_loss: 0.7952 - val_accuracy: 0.7266
Epoch 5/10
1123/1123 [==============================] - 67s 60ms/step - loss: 0.7584 - accuracy: 0.7397 - val_loss: 0.7930 - val_accuracy: 0.7255
Epoch 6/10
1123/1123 [==============================] - 91s 81ms/step - loss: 0.7404 - accuracy: 0.7461 - val_loss: 0.7806 - val_accuracy: 0.7336
Epoch 7/10
1123/1123 [==============================] - 287s 256ms/step - loss: 0.7253 - accuracy: 0.7510 - val_loss: 0.7800